In [25]:
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd

In [26]:
# Load the data
demand_distribution_df = pd.read_csv("/Users/mahinbindra/Downloads/distributions (1).csv")
cost_matrix_df = pd.read_csv("/Users/mahinbindra/Downloads/cost_matrix (1).csv")

In [27]:
# Initialize the model
model = Model("Inventory_Transshipment")

In [28]:
# Number of stores
num_stores = len(cost_matrix_df)

In [29]:
# Set up scenario probabilities and demands for each scenario
num_scenarios = 10  # You should define this based on your scenario data
prob_scenario = 1 / num_scenarios  # Equal probability for each scenario as a placeholder
scenario_demands = np.random.choice(demand_distribution_df['Mean'], num_scenarios)  # Placeholder for scenario demands

In [30]:
# Cost parameters
ordering_cost = 24.44
holding_cost = 25.55  # This may represent the combination of overage and underage costs per unit
cost_matrix = cost_matrix_df.values

In [31]:
# First stage variables
order_qty = model.addVars(num_stores, vtype=GRB.CONTINUOUS, name="Order_Quantity")

In [32]:
# Second stage variables
overage = model.addVars(num_stores, num_scenarios, vtype=GRB.CONTINUOUS, name="Overage")
underage = model.addVars(num_stores, num_scenarios, vtype=GRB.CONTINUOUS, name="Underage")
transshipment = model.addVars(num_stores, num_stores, num_scenarios, vtype=GRB.CONTINUOUS, name="Transshipment")

In [33]:
# Objective Function
model.setObjective(
    (1 / num_scenarios) * quicksum(
        ordering_cost * order_qty[i] +
        holding_cost * overage[i, k] +
        holding_cost * underage[i, k] +
        quicksum(cost_matrix[i, j] * transshipment[i, j, k] for j in range(num_stores) if i != j)
        for i in range(num_stores) for k in range(num_scenarios)),
    GRB.MINIMIZE)

In [34]:
# First stage constraints
for i in range(num_stores):
    model.addConstr(order_qty[i] >= 0, name=f"Non_Negativity_y_{i}")

# Second stage constraints
for k in range(num_scenarios):
    for i in range(num_stores):
        # Overage and underage constraints
        model.addConstr(overage[i, k] >= order_qty[i] - scenario_demands[k], name=f"Overage_{i}_{k}")
        model.addConstr(underage[i, k] >= scenario_demands[k] - order_qty[i], name=f"Underage_{i}_{k}")

        # Transshipment constraints
        for j in range(num_stores):
            if i != j:
                model.addConstr(transshipment[i, j, k] <= order_qty[i] - overage[i, k], name=f"Transshipment_{i}_{j}_{k}")

In [35]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2415 rows, 2565 columns and 6915 nonzeros
Model fingerprint: 0x6010b5c8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 2115 rows and 2250 columns
Presolve time: 0.01s
Presolved: 300 rows, 465 columns, 750 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.580639e+04   0.000000e+00      0s
     150    3.9273994e+04   0.000000e+00   0.000000e+00      0s

Solved in 150 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.927399361e+04


In [36]:
# Output solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    # Retrieving and printing the decision variables may be added here as needed
else:
    print("No optimal solution found or there was an issue with the optimization.")

Optimal solution found.


In [37]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

# Load the demand distribution data
demand_distribution_df = pd.read_csv("/Users/mahinbindra/Downloads/distributions (1).csv")

# Parameters
num_stores = len(demand_distribution_df)
num_trials = 50
num_scenarios_per_trial = 100
ordering_cost = 24.44
per_unit_overage_cost = 25.55  # Cost per unit of unsold inventory
per_unit_underage_cost = 25.55  # Opportunity cost per unit of excess demand

# Function to run a single trial
def run_trial():
    model = Model("Inventory_Optimization_No_Transshipment")

    # Generate demand scenarios
    demand_scenarios = np.array([
        np.random.binomial(n=row['n'], p=row['p'], size=num_scenarios_per_trial)
        for _, row in demand_distribution_df.iterrows()
    ])

    # Decision Variables
    order_qty = model.addVars(num_stores, vtype=GRB.CONTINUOUS, name="Order_Quantity")
    overage = model.addVars(num_stores, num_scenarios_per_trial, vtype=GRB.CONTINUOUS, name="Overage")
    underage = model.addVars(num_stores, num_scenarios_per_trial, vtype=GRB.CONTINUOUS, name="Underage")

    # Objective Function
    model.setObjective(
        (1 / num_scenarios_per_trial) * quicksum(
            ordering_cost * order_qty[i] +
            per_unit_overage_cost * overage[i, s] +
            per_unit_underage_cost * underage[i, s]
            for i in range(num_stores) for s in range(num_scenarios_per_trial)),
        GRB.MINIMIZE)

    # Constraints
    for i in range(num_stores):
        for s in range(num_scenarios_per_trial):
            model.addConstr(overage[i, s] >= order_qty[i] - demand_scenarios[i, s], name=f"Overage_{i}_{s}")
            model.addConstr(underage[i, s] >= demand_scenarios[i, s] - order_qty[i], name=f"Underage_{i}_{s}")

    # Solve the model
    model.optimize()

    # Check if the model has been solved to optimality
    if model.status == GRB.OPTIMAL:
        # Record the optimal cost for this trial
        return model.objVal
    else:
        raise Exception("Optimization was unsuccessful or did not reach optimality.")

# Run all trials and record the costs
trial_costs = [run_trial() for _ in range(num_trials)]

# Calculate the average cost associated with the optimal plan
average_cost = np.mean(trial_costs)
print(f"The average cost associated with the optimal plan over {num_trials} trials is: {average_cost}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3000 rows, 3015 columns and 6000 nonzeros
Model fingerprint: 0x8369159b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve time: 0.00s
Presolved: 3000 rows, 3015 columns, 6000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.428400e+04   0.000000e+00      0s
     274    3.3981850e+04   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 1.1810000e+00      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     2